In [124]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

In [138]:
inventory = pd.read_csv('MS-b1-inventory.csv')
supply = pd.read_csv('MS-b1-supply.csv')
sell = pd.read_csv('MS-b1-sell.csv')
supply['date'] = pd.to_datetime(supply['date'], format='%Y-%m-%d')
inventory['date'] = pd.to_datetime(inventory['date'], format='%Y-%m-%d')

In [114]:
print(inventory)

          date    apple     pen
0   2006-01-31    12157     811
1   2006-02-28    29859    2280
2   2006-03-31    42135    3317
3   2006-04-30    55235    4094
4   2006-05-31    66963    5158
..         ...      ...     ...
115 2015-08-31  1518055  118479
116 2015-09-30  1532120  119640
117 2015-10-31  1543664  120608
118 2015-11-30  1557415  121734
119 2015-12-31  1569425  122601

[120 rows x 3 columns]


### 1.1 Состояние склада на каждый день

In [126]:
sell_conv = sell
sell_conv['sku_num'] = sell['sku_num'].apply(lambda x: 1 if x[:8] == "MS-b1-ap" else 2)
sell_conv["pen"] = (sell_conv['sku_num'] == 2).astype(int)
sell_conv["apple"] = (sell_conv['sku_num'] == 1).astype(int)
daily_s = sell_conv.groupby('date').sum()
daily_s['date'] = daily_s.index
daily_s = daily_s.reset_index(drop=True)
daily_s = daily_s.drop(columns=["sku_num"])
daily_s = daily_s.loc[:,['date','apple','pen']]
daily_s['date'] = pd.to_datetime(daily_s['date'], format='%Y-%m-%d')
print(daily_s) #продажи на каждый день

           date  apple  pen
0    2006-01-01   1815  156
1    2006-01-02   1862  143
2    2006-01-03   1880  171
3    2006-01-04   1797  153
4    2006-01-05   1942  133
...         ...    ...  ...
3647 2015-12-27   1831  152
3648 2015-12-28   1797  160
3649 2015-12-29   1840  139
3650 2015-12-30   1820  140
3651 2015-12-31   1884  124

[3652 rows x 3 columns]


In [139]:
daily = pd.DataFrame(0, index=np.arange(len(daily_s[30:])), columns=['date', 'apple', 'pen'])
daily["date"] = list(daily_s[30:]["date"])
daily.loc[0] = inventory.loc[0]
for i in list(daily_s["date"][1:-30]):
    j = i +  timedelta(days=30)
    apples_in = 0
    pen_in = 0
    if j in list(supply["date"]):
        apples_in = int(supply[supply['date']==j]['apple'])
        pen_in = int(supply[supply['date']==j]['pen'])
    apple_vol = int(daily[daily["date"] == (j - timedelta(1))]['apple']) - int(daily_s[daily_s["date"] == j]['apple']) + apples_in
    pen_vol = int(daily[daily["date"] == (j - timedelta(1))]['pen']) - int(daily_s[daily_s["date"] == j]['pen']) + pen_in
    daily[daily["date"] == j] = [j,apple_vol,pen_vol]
    if j in list(inventory['date']):
        apples_vol = int(inventory[inventory['date']==j]["apple"]) 
        pen_vol =  int(inventory[inventory['date']==j]["pen"]) 
        daily[daily["date"] == j] = [j, apples_vol, pen_vol]

In [141]:
daily[:15] #состояние на каждый день

,date,apple,pen
0,2006-01-31,12157,811
1,2006-02-01,45336,3413
2,2006-02-02,43452,3284
3,2006-02-03,41633,3153
4,2006-02-04,39781,3015
5,2006-02-05,37918,2860
6,2006-02-06,36068,2701
7,2006-02-07,34274,2552
8,2006-02-08,32401,2415
9,2006-02-09,30517,2276


### 1.2 Три лучших месяца с точки зрения самых эффективных продаж

In [142]:
month_s = daily_s
month_s.index = month_s["date"]
month_s = month_s.drop(columns=["date"])
month_s = month_s.groupby(by=[month_s.index.month, month_s.index.year]).sum()
best_month_s_pen = month_s.sort_values(by='pen')[-3:]
best_month_s_apple = month_s.sort_values(by='apple')[-3:]
print("Best months for pen:", list(month_s.sort_values(by='pen')[-3:].index))
print("The pen volume: ", list(best_month_s_pen['pen']))
print()
print("Best months for apple:", list(month_s.sort_values(by='apple')[-3:].index))
print("The apple volume: ", list(best_month_s_apple['apple']))

Best months for pen: [(12, 2010), (8, 2009), (1, 2014)]
The pen volume:  [4564, 4599, 4617]

Best months for apple: [(5, 2013), (7, 2012), (1, 2011)]
The apple volume:  [58434, 58438, 58512]


#### Лучшие месяцы по продажам ручек: <br> 1) Декабрь 2010 - 4564 шт. <br> 2) Август 2009 - 4599 шт. <br> 3) Январь 2014 - 4617 шт.<br>
#### Лучшие месяцы по продажам яблок: <br> 1) Май 2013 - 58434 шт. <br> 2) Июль 2012 - 58438 шт. <br> 3) Январь 2011 - 58512 шт.

### 2.1 Месячные данные о количестве сворованного товара

In [158]:
month_st = pd.DataFrame(columns=['date', 'apple', 'pen'])

wr_daily = pd.DataFrame(0, index=np.arange(len(daily_s[30:])), columns=['date', 'apple', 'pen'])
wr_daily["date"] = list(daily_s[30:]["date"])
wr_daily.loc[0] = inventory.loc[0]

for i in list(daily_s["date"][1:-30]):
    j = i +  timedelta(days=30)
    apples_in = 0
    pen_in = 0
    if j in list(supply["date"]):
        apples_in = int(supply[supply['date']==j]['apple'])
        pen_in = int(supply[supply['date']==j]['pen'])
    apple_vol = int(wr_daily[wr_daily["date"] == (j - timedelta(1))]['apple']) - int(daily_s[daily_s["date"] == j]['apple']) + apples_in
    pen_vol = int(wr_daily[wr_daily["date"] == (j - timedelta(1))]['pen']) - int(daily_s[daily_s["date"] == j]['pen']) + pen_in
    wr_daily[wr_daily["date"] == j] = [j,apple_vol,pen_vol]
    if j in list(inventory['date']):
        apple_st = int(wr_daily[wr_daily["date"] == (j)]['apple']) - int(inventory[inventory['date']==j]["apple"]) 
        pen_st = int(wr_daily[wr_daily["date"] == (j)]['pen']) - int(inventory[inventory['date']==j]["pen"]) 
        month_st.loc[len(month_st.index)] = [j,apple_st,pen_st] 
        wr_daily[wr_daily["date"] == j] = daily[daily["date"] == j]

In [152]:
month_st[:15]

,date,apple,pen
0,2006-02-28,6,6
1,2006-03-31,7,6
2,2006-04-30,6,14
3,2006-05-31,8,1
4,2006-06-30,7,9
5,2006-07-31,10,12
6,2006-08-31,3,5
7,2006-09-30,6,4
8,2006-10-31,1,9
9,2006-11-30,6,7


### 1.2 Три худших месяца с точки зрения краж

In [164]:
month_stt = month_st
month_stt.index = month_st["date"]
month_stt = month_stt.drop(columns=["date"])
month_stt = month_stt.groupby(by=month_stt.index.year).sum()
best_month_st_pen = month_stt.sort_values(by='pen')[-3:]
best_month_st_apple = month_stt.sort_values(by='apple')[-3:]
print("Worst months for pen:", list(month_stt.sort_values(by='pen')[-3:].index))
print("The pen volume: ", list(best_month_st_pen['pen']))
print()
print("Worst months for apple:", list(month_stt.sort_values(by='apple')[-3:].index))
print("The apple volume: ", list(best_month_st_apple['apple']))

Worst months for pen: [2009, 2010, 2011]
The pen volume:  [96, 96, 100]

Worst months for apple: [2012, 2011, 2014]
The apple volume:  [82, 88, 90]


#### Худшие месяцы по кражам ручек: <br> 1) 2009 - 96 шт. <br> 2) 2010 - 96 шт. <br> 3) 2011 - 10 шт.<br>
#### Худшие месяцы по кражам яблок: <br> 1) 2012 - 82 шт. <br> 2) 2011 - 88 шт. <br> 3) 2014 - 90 шт.